In [3]:
# notebooks/phase2_ablation_analysis.ipynb

import pandas as pd
import wandb
from pathlib import Path

# Configure pandas for better display
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 100)



In [4]:
# --- 1. Fetch Data from W&B ---
print("Fetching ablation suite runs from Weights & Biases...")
api = wandb.Api()

# --- FIX: Robust Query ---
# The previous query failed. A more robust way is to fetch all runs and filter locally,
# or to ensure the filter keys are exactly correct. The "config.key" format is correct.
# Let's try the correct filter first.
try:
    runs = api.runs(
        "mohan-rangan-research-nosfera/Jormungandr-Semantica",
        filters={
            "config.method": "jormungandr",
            # We filter on a config value that IS present: representation
            "config.representation": {"$in": ["direct", "wavelet", "acmw"]},
            "state": "finished"
        }
    )
    print(f"Found {len(runs)} runs to analyze.")
except Exception as e:
    print(f"W&B API query failed with error: {e}. Check filter syntax.")
    runs = [] # Set to empty list to avoid crashing the rest of the script



Fetching ablation suite runs from Weights & Biases...
Found 33 runs to analyze.


In [5]:
# --- 2. Process Runs into a DataFrame ---
summary_list = []
if runs:
    for run in runs:
        summary_list.append({
            "representation": run.config.get("representation"),
            "seed": run.config.get("seed"),
            "ARI": run.summary.get("ARI"),
            "runtime_seconds": run.summary.get("runtime_seconds")
        })

results_df = pd.DataFrame(summary_list)



In [6]:
# --- 3. Data Cleaning and Initial Inspection ---
if not results_df.empty:
    results_df = results_df.dropna().sort_values(by=["representation", "seed"]).reset_index(drop=True)
    print("\nRaw Results DataFrame:")
    display(results_df)

    # --- 4. Generate Publication-Ready Summary Table ---
    print("\n--- Ablation Results: Mean ARI ± Std. Dev. ---")
    summary_table = results_df.groupby('representation')['ARI'].agg(['mean', 'std', 'count'])
    
    rep_order = ['direct', 'wavelet', 'acmw']
    summary_table = summary_table.reindex(rep_order)

    formatted_table = pd.DataFrame(index=summary_table.index)
    formatted_table['Mean ARI'] = summary_table['mean']
    formatted_table['Std. Dev.'] = summary_table['std']
    formatted_table['Run Count'] = summary_table['count']
    formatted_table['Formatted ARI'] = summary_table.apply(
        lambda row: f"{row['mean']:.4f} ± {row['std']:.4f}", axis=1
    )
    display(formatted_table)

    # --- 5. Save the Markdown table for easy inclusion in the paper ---
    markdown_output = formatted_table[['Formatted ARI']].to_markdown()
    print("\nMarkdown for Paper (Table 8.1):")
    print(markdown_output)
    
    output_path = Path("../data/phase2_ablation_results.csv")
    results_df.to_csv(output_path, index=False)
    print(f"\nRaw data for this analysis saved to {output_path}")
else:
    print("\nNo runs found or W&B query failed. Cannot generate analysis.")


Raw Results DataFrame:


,representation,seed,ARI,runtime_seconds
0,acmw,42,0.2384,214.4349
1,acmw,42,0.2384,197.6020
2,acmw,43,0.2574,195.8597
3,acmw,44,0.2539,196.5849
4,acmw,45,0.2409,1859.7169
5,acmw,46,0.1953,197.9722
6,acmw,47,0.2655,202.1379
7,acmw,48,0.2555,198.6147
8,acmw,49,0.2610,198.7630
9,acmw,50,0.2574,194.4923



--- Ablation Results: Mean ARI ± Std. Dev. ---


,Mean ARI,Std. Dev.,Run Count,Formatted ARI
representation,,,,
direct,0.3875,0.0124,10,0.3875 ± 0.0124
wavelet,0.3843,0.0219,12,0.3843 ± 0.0219
acmw,0.2478,0.0198,11,0.2478 ± 0.0198



Markdown for Paper (Table 8.1):
| representation   | Formatted ARI   |
|:-----------------|:----------------|
| direct           | 0.3875 ± 0.0124 |
| wavelet          | 0.3843 ± 0.0219 |
| acmw             | 0.2478 ± 0.0198 |

Raw data for this analysis saved to ../data/phase2_ablation_results.csv
